# Noncontextual Electronic Structure

How to solve $H_{noncontextual}$ given the following partition:

$$H_{full} = H_\mathrm{noncontextual} + H_\mathrm{context}$$

# First we need a Hamiltonian

In [1]:
import os
import json

cwd = os.getcwd()
notebook_dir =  os.path.dirname(cwd)
symmer_dir = os.path.dirname(notebook_dir)
test_dir = os.path.join(symmer_dir, 'tests')
ham_data_dir = os.path.join(test_dir, 'hamiltonian_data')

if not os.path.isdir(ham_data_dir):
    raise ValueError('cannot find data dir')

In [2]:
# os.listdir(ham_data_dir)

In [3]:
filename = 'B_STO-3G_DOUBLET_JW.json'#'H2O_STO-3G_SINGLET_BK.json'

if filename not in os.listdir(ham_data_dir):
    raise ValueError('unknown file')
    
with open(os.path.join(ham_data_dir, filename), 'r') as infile:
    data_dict = json.load(infile)

In [4]:
from symmer.symplectic import PauliwordOp
H = PauliwordOp.from_dictionary(data_dict['hamiltonian'])
H.n_qubits

10

# Solving the noncontexutal problem!

In [5]:
from symmer.projection import ContextualSubspace

cs_vqe = ContextualSubspace(H, noncontextual_strategy='SingleSweep_magnitude')

Noncontextual Hamiltonians have a very specific form, namely their terms $\mathcal{T}$ may be decomposed as
\begin{equation}
    \mathcal{T} = \mathcal{S} \cup \mathcal{C}_1 \cup \dots \cup \mathcal{C}_M 
\end{equation}
where $\mathcal{S}$ contains the terms that commute globally (i.e. the noncontextual symmetries) and a collection of cliques with respect to commutation.

In [6]:
cs_vqe.noncontextual_operator.decomposed

{'symmetry': -15.808+0.000j IIIIIIIIII +
  0.050+0.000j IIIIIIIIIZ +
  0.050+0.000j IIIIIIIIZI +
  0.147+0.000j IIIIIIIIZZ +
  0.050+0.000j IIIIIIIZII +
  0.123+0.000j IIIIIIIZIZ +
  0.131+0.000j IIIIIIIZZI +
  0.050+0.000j IIIIIIZIII +
  0.131+0.000j IIIIIIZIIZ +
  0.123+0.000j IIIIIIZIZI +
  0.147+0.000j IIIIIIZZII +
  0.050+0.000j IIIIIZIIII +
  0.123+0.000j IIIIIZIIIZ +
  0.131+0.000j IIIIIZIIZI +
  0.123+0.000j IIIIIZIZII +
  0.131+0.000j IIIIIZZIII +
  0.050+0.000j IIIIZIIIII +
  0.131+0.000j IIIIZIIIIZ +
  0.123+0.000j IIIIZIIIZI +
  0.131+0.000j IIIIZIIZII +
  0.123+0.000j IIIIZIZIII +
  0.147+0.000j IIIIZZIIII +
  0.198+0.000j IIZIIIIIII +
  0.133+0.000j IIZIIIIIIZ +
  0.105+0.000j IIZIIIIIZI +
  0.133+0.000j IIZIIIIZII +
  0.105+0.000j IIZIIIZIII +
  0.133+0.000j IIZIIZIIII +
  0.105+0.000j IIZIZIIIII +
  0.171+0.000j IZIZIIIIII +
  3.977+0.000j ZIIIIIIIII +
  0.186+0.000j ZIIIIIIIIZ +
  0.180+0.000j ZIIIIIIIZI +
  0.186+0.000j ZIIIIIIZII +
  0.180+0.000j ZIIIIIZIII +
  0.186

In [7]:
from symmer.symplectic import NoncontextualOp
H_noncon = NoncontextualOp.from_PauliwordOp(cs_vqe.noncontextual_operator)
H_noncon

-15.808+0.000j IIIIIIIIII +
 0.050+0.000j IIIIIIIIIZ +
 0.050+0.000j IIIIIIIIZI +
 0.147+0.000j IIIIIIIIZZ +
 0.050+0.000j IIIIIIIZII +
 0.123+0.000j IIIIIIIZIZ +
 0.131+0.000j IIIIIIIZZI +
 0.050+0.000j IIIIIIZIII +
 0.131+0.000j IIIIIIZIIZ +
 0.123+0.000j IIIIIIZIZI +
 0.147+0.000j IIIIIIZZII +
 0.050+0.000j IIIIIZIIII +
 0.123+0.000j IIIIIZIIIZ +
 0.131+0.000j IIIIIZIIZI +
 0.123+0.000j IIIIIZIZII +
 0.131+0.000j IIIIIZZIII +
 0.050+0.000j IIIIZIIIII +
 0.131+0.000j IIIIZIIIIZ +
 0.123+0.000j IIIIZIIIZI +
 0.131+0.000j IIIIZIIZII +
 0.123+0.000j IIIIZIZIII +
 0.147+0.000j IIIIZZIIII +
 0.198+0.000j IIIZIIIIII +
 0.105+0.000j IIIZIIIIIZ +
 0.133+0.000j IIIZIIIIZI +
 0.105+0.000j IIIZIIIZII +
 0.133+0.000j IIIZIIZIII +
 0.105+0.000j IIIZIZIIII +
 0.133+0.000j IIIZZIIIII +
 0.198+0.000j IIZIIIIIII +
 0.133+0.000j IIZIIIIIIZ +
 0.105+0.000j IIZIIIIIZI +
 0.133+0.000j IIZIIIIZII +
 0.105+0.000j IIZIIIZIII +
 0.133+0.000j IIZIIZIIII +
 0.105+0.000j IIZIZIIIII +
 0.132+0.000j IIZZIIIIII +


# Solving the noncontexutal problem!

All terms in $H_{noncon}$ can be generated under the jordan product: $P_{a}\cdot P_{b} = \frac{1}{2} \{P_{a}, P_{b} \}$. This ensures only one term from the anticommuting set can be used, as joint assignement to anticommuting Pauli operator is not possible!

The expectation value of $H_{noncon}$ is therefore fixed by  and$\vec{r}$

Identifying a generating set $\mathcal{G}$ for the symmetry terms $\mathcal{S}$ and constructing the clique operator $A(\vec{r}) = \sum_{i=1}^M r_i C_i$ for clique representatives $C_i \in \mathcal{C}_i$ and coefficients $\vec{r} \in \mathbb{R}^M$ satisfying $|\vec{r}|=1$ allows us to rewrite

\begin{equation}
    H_\mathrm{noncon} = \sum_{P \in \overline{\mathcal{G}}} \bigg(h_{P}^\prime + \sum_{i=1}^M h_{P,i} C_i \bigg) P,
\end{equation}

and yields a classical objective function over parameters $\vec{\nu} \in \{\pm 1\}^{|\mathcal{G}|}$ and $\vec{r} \in \mathbb{R}^M$ for the noncontetual energy expectation value:

\begin{equation}\label{classical_objective}
\begin{aligned}
    \eta(\vec{\nu}, \vec{r}) 
    :={} & {\langle H_\mathrm{noncon} \rangle_{(\vec{\nu}, \vec{r})}} \\
    ={} & \sum_{P \in \overline{\mathcal{G}}} \bigg(h_{P}^\prime + \sum_{i=1}^M h_{P,i} \langle{C_i}\rangle_{(\vec{\nu}, \vec{r})} \bigg) \langle{P}\rangle_{(\vec{\nu}, \vec{r})} \\
    ={} & \sum_{P \in \overline{\mathcal{G}}} \bigg(h_{P}^\prime + \sum_{i=1}^M h_{P,i} r_i \bigg) \prod_{G \in \mathcal{G}_{P}} \nu_{f(G)}.
\end{aligned}
\end{equation}

### Different optimization strategies to find the ground state!

In [8]:
# %%timeit
H_noncon.solve(strategy='brute_force')
H_noncon.energy

-24.14898859885364

In [9]:
# %%timeit
H_noncon.solve(strategy='binary_relaxation')
H_noncon.energy

-24.148988598843847

In [10]:
# PUSO = Polynomial unconstrained spin Optimization
# QUSO: Quadratic Unconstrained Spin Optimization

In [11]:
# %%timeit
H_noncon.solve(strategy='brute_force_PUSO')
H_noncon.energy

-24.075497798801486

In [12]:
# %%timeit
H_noncon.solve(strategy='brute_force_QUSO')
H_noncon.energy

-24.148988598853506

In [13]:
# %%timeit
H_noncon.solve(strategy='annealing_PUSO', num_anneals=100)
H_noncon.energy

-24.14898859885344

In [14]:
# %%timeit
H_noncon.solve(strategy='annealing_QUSO', num_anneals=100)
H_noncon.energy

-14.59161692426567

In [15]:
# # brute_force_PUSO
# energy, nu, r = H_noncon._energy_via_brute_force_xUSO(x='P')
# energy

In [16]:
# # brute_force_QUSO
# energy, nu, r = H_noncon._energy_via_brute_force_xUSO(x='Q')
# energy

In [17]:
# # annealing_QUSO
# energy, nu, r = H_noncon._energy_via_annealing_xUSO(x='Q', num_anneals=1000)
# energy

In [18]:
# # annealing_PUSO
# energy, nu, r = H_noncon._energy_via_annealing_xUSO(x='P', num_anneals=100)
# energy